##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기 전에

시작하기 전에 다음을 실행하여 환경이 올바르게 설정되었는지 확인하세요. 인사말이 표시되지 않으면 [설치](../install.md) 가이드에서 지침을 참조하세요. 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**참고**: 이 Colab은 <code>tensorflow_federated</code> pip 패키지의 <a>최신 릴리즈 버전</a>에서 동작하는 것으로 확인되었지만 Tensorflow Federated 프로젝트는 아직 시험판 개발 중이며 `main`에서 동작하지 않을 수 있습니다.

# 고유한 페더레이션 학습 알고리즘 구축하기

[이미지 분류](federated_learning_for_image_classification.ipynb) 및 [텍스트 생성](federated_learning_for_text_generation.ipynb) 튜토리얼에서는 페더레이션 학습(FL)을 위한 모델 및 데이터 파이프라인을 설정하는 방법을 배웠고 TFF의 `tff.learning` API 계층을 통해 페더레이션 학습을 수행했습니다.

이것은 FL 연구와 관련하여 빙산의 일각에 불과합니다. 이 튜토리얼에서는 `tff.learning` API를 따르지 *않고* 페더레이션 학습 알고리즘을 구현하는 방법을 논의합니다. 여기서는 다음을 목표로 합니다.

**목표**

- 페더레이션 학습 알고리즘의 일반적인 구조를 이해합니다.
- TFF의 *페더레이션 코어*를 탐색합니다.
- Federated Core를 사용하여 Federated Averaging을 직접 구현합니다.

이 튜토리얼은 자체적으로 진행할 수 있지만 먼저 [이미지 분류](federated_learning_for_image_classification.ipynb) 및 [텍스트 생성](federated_learning_for_text_generation.ipynb) 튜토리얼을 읽어보는 것이 좋습니다.


## 입력 데이터 준비하기

먼저 TFF에 포함된 EMNIST 데이터세트를 로드하고 전처리합니다. 자세한 내용은 [이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼을 참조하세요.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

데이터세트를 모델에 제공하기 위해 데이터를 평면화하고 각 예제를 `(flattened_image_vector, label)` 형식의 튜플로 변환합니다.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

이제 소수의 클라이언트를 선택하고 위의 전처리를 데이터세트에 적용합니다.

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 모델 준비하기

[이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼에서와 동일한 모델을 사용합니다. 이 모델(`tf.keras`를 통해 구현됨)에는 하나의 숨겨진 레이어, 그리고 이어서 소프트맥스 레이어가 있습니다.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

TFF에서 이 모델을 사용하기 위해 Keras 모델을 [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model)로 래핑합니다. 이를 통해 TFF 내에서 모델의 [정방향 전달](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass)을 수행하고 [모델 출력을 추출](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)할 수 있습니다. 자세한 내용은 [이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼을 참조하세요.

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

여기서는 `tf.keras`를 사용하여 `tff.learning.Model`을 생성하지만 TFF는 훨씬 더 많은 일반 모델을 지원합니다. 이러한 모델에는 모델 가중치를 캡처하는 다음과 같은 관련 속성이 있습니다.

- `trainable_variables`: 학습 가능한 레이어에 해당하는 텐서의 이터러블입니다.
- `non_trainable_variables`: 학습할 수 없는 레이어에 해당하는 텐서의 이터러블입니다.

여기서는 목적상 `trainable_variables`만 사용합니다(우리 모델에는 이것만 있으므로!).

# 고유한 페더레이션 학습 알고리즘 구축하기

`tff.learning` API를 사용하면 Federated Averaging의 많은 변형을 생성할 수 있지만 이 프레임워크에 깔끔하게 맞지 않는 다른 페더레이션 알고리즘이 있습니다. 예를 들어 정규화, 클리핑 또는 [페더레이션 GAN 학습](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans)과 같은 더 복잡한 알고리즘을 추가할 수도 있습니다. 아니면 [페더레이션 분석](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html)에 관심이 있을 수도 있습니다.

이러한 고급 알고리즘의 경우 TFF를 사용하여 자체 사용자 지정 알고리즘을 작성해야 합니다. 많은 경우 페더레이션 알고리즘에는 4가지 주요 구성 요소가 있습니다.

1. 서버-클라이언트 브로드캐스트 단계
2. 로컬 클라이언트 업데이트 단계
3. 클라이언트-서버 업로드 단계
4. 서버 업데이트 단계

TFF에서는 일반적으로 페더레이션 알고리즘을 [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess)(나머지 부분에서 `IterativeProcess`라고 함)로 나타냅니다. 이것은 `initialize` 및 `next` 함수를 포함하는 클래스입니다. 여기서 `initialize`는 서버를 초기화하는 데 사용되며 `next`는 페더레이션 알고리즘의 한 통신 라운드를 수행합니다. FedAvg에 대한 반복 프로세스가 어떤 모습인지 그 골격을 작성해 보겠습니다.

먼저 `tff.learning.Model`을 생성하고 학습 가능한 가중치를 반환하는 초기화 함수가 있습니다.

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

이 함수는 문제 없어 보이지만 "TFF 계산"이 되도록 약간 수정해야 합니다. 이 내용은 나중에 알아볼 것입니다.

또한, 우리는 `next_fn`를 스케치하려고 합니다.

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

이 네 가지 구성 요소를 개별적으로 구현하는 데 중점을 둘 것입니다. 먼저 순수 TensorFlow에서 구현할 수 있는 부분, 즉 클라이언트 및 서버 업데이트 단계에 초점을 맞출 것입니다.


## TensorFlow 블록 

### 클라이언트 업데이트

`tff.learning.Model`을 사용하여 기본적으로 TensorFlow 모델을 훈련하는 것과 동일한 방식으로 클라이언트 훈련을 수행합니다. 특히, 우리는 `tf.GradientTape`를 사용하여 데이터 배치에 대한 그레디언트를 계산한 다음 `client_optimizer`를 사용하여 이러한 그레디언트를 적용합니다. 우리는 훈련 가능한 가중치에만 초점을 맞춥니다.


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### 서버 업데이트

FedAvg에 대한 서버 업데이트는 클라이언트 업데이트보다 간단합니다. 우리는 단순히 서버 모델 가중치를 클라이언트 모델 가중치의 평균으로 바꾸는 "바닐라" 페더레이션 평균화를 구현할 것입니다. 다시 말하지만, 우리는 훈련 가능한 가중치에만 초점을 맞춥니다.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

단순히 `mean_client_weights`를 반환하여 스니펫을 단순화할 수 있습니다. 그러나 Federated Averaging의 고급 구현에서는 모멘텀 또는 적응성과 같은 보다 정교한 기술과 함께 `mean_client_weights`를 사용합니다.

**과제**: model_weights 및 mean_client_weights의 중간점이 되도록 서버 가중치를 업데이트하는 `server_update` 버전을 구현합니다(참고: 이러한 종류의 "중간점" 접근 방식은 [Lookahead 옵티마이저](https://arxiv.org/abs/1907.08610)에 대한 최근의 작업과 유사합니다!).

지금까지 순수한 TensorFlow 코드만 작성했습니다. TFF를 사용하면 이미 익숙한 TensorFlow 코드를 많이 사용할 수 있으므로 이는 의도적으로 설계된 것입니다. 그러나 이제 <em>오케스트레이션 로직</em>, 즉 서버가 클라이언트에 브로드캐스트하는 내용과 클라이언트가 서버에 업로드하는 내용을 지정하는 로직을 지정해야 합니다.

이를 위해서는 TFF의 *페더레이션 코어*가 필요합니다.

# 페더레이션 코어 소개

페더레이션 코어(FC)는 `tff.learning` API의 기반 역할을 하는 하위 수준 인터페이스 집합입니다. 그러나 이러한 인터페이스는 학습에만 국한되지 않습니다. 실제로 분산 데이터에 대한 분석 및 기타 많은 계산에 사용할 수 있습니다.

상위 수준에서 페더레이션 코어는 간결하게 표현된 프로그램 로직을 사용하여 TensorFlow 코드를 분산 통신 연산자(예: 분산 합계 및 브로드캐스트)와 결합할 수 있는 개발 환경입니다. 목표는 시스템 구현 세부 사항(예: 지점 간 네트워크 메시지 교환 지정)을 요구하지 않고 연구자와 실무자에게 시스템의 분산 통신을 신속하게 제어할 수 있도록 하는 것입니다.

한 가지 요점은 TFF가 개인 정보 보호를 위해 설계되었다는 것입니다. 따라서 중앙 집중식 서버 위치에서 원치 않는 데이터 축적을 방지하기 위해 데이터가 있는 위치를 명시적으로 제어할 수 있습니다.

## 페더레이션 데이터

TFF의 핵심 개념은 "페더레이션 데이터"이며, 이는 분산 시스템의 장치 그룹(예: 클라이언트 데이터세트 또는 서버 모델 가중치)에 걸쳐 호스팅되는 데이터 항목의 집합을 나타냅니다. 우리는 모든 장치에서 데이터 항목의 전체 컬렉션을 단일 *페더레이션 값*으로 모델링합니다.

예를 들어, 센서의 온도를 나타내는 부동 소수점이 있는 클라이언트 장치가 있다고 가정합니다. 우리는 이것을 *페더레이션 부동 소수점*으로 표현할 수 있습니다.

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

페더레이션 유형은 해당 구성원 구성 요소(예: `tf.float32`)의 유형 `T`와 장치 그룹 `G`로 지정됩니다. `G`가 `tff.CLIENTS` 또는 `tff.SERVER`인 경우에 중점을 둘 것입니다. 이러한 페더레이션 유형은 아래와 같이 `{T}@G`로 표시됩니다.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

게재 위치에 관심이 많은 이유는 무엇입니까? TFF의 핵심 목표는 실제 분산 시스템에 배포할 수 있는 코드를 작성할 수 있도록 하는 것입니다. 즉, 장치의 하위 집합이 어떤 코드를 실행하고 다른 데이터 조각이 어디에 있는지 추론하는 것이 중요합니다.

TFF는 *데이터*, 데이터가 *배치되는 위치* 및 데이터가 *변환*되는 방법의 세 가지에 중점을 둡니다. 처음 두 개는 페더레이션 유형으로 캡슐화되고 마지막 두 개는 *페더레이션 계산*에 캡슐화됩니다.

## 페더레이션 계산

TFF는 기본 단위가 *페더레이션 계산*인 강력한 형식의 함수형 프로그래밍 환경입니다. 이들은 페더레이션 값을 입력으로 받아들이고 페더레이션 값을 출력으로 반환하는 논리 조각입니다.

예를 들어 클라이언트 센서의 온도를 평균화하고 싶다고 가정합니다. 다음을 정의할 수 있습니다(페더레이션 부동 소수점 사용).

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

TensorFlow의 `tf.function` 데코레이터와 어떻게 다른지 물어볼 수 있습니다. 핵심적인 대답은 `tff.federated_computation`에 의해 생성된 코드가 TensorFlow도 아니고 Python 코드도 아니라는 것입니다. 내부 플랫폼 독립적인 *글루 언어*로 된 분산 시스템의 사양입니다.

복잡하게 들릴 수 있지만 TFF 계산은 잘 정의된 형식 서명이 있는 함수로 생각할 수 있습니다. 이러한 유형 서명은 직접 쿼리할 수 있습니다.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

이 `tff.federated_computation`은 페더레이션 유형 `<float>@CLIENTS`의 인수를 받아들이고 페더레이션 유형 `<float>@SERVER`의 값을 반환합니다. 페더레이션 계산은 서버에서 클라이언트로, 클라이언트에서 클라이언트로 또는 서버에서 서버로 이동할 수도 있습니다. 페더레이션 계산은 유형 서명이 일치하는 한 일반 함수처럼 구성할 수도 있습니다.

개발을 지원하기 위해 TFF를 사용하면 `tff.federated_computation`을 Python 함수로 호출할 수 있습니다. 예를 들어 다음을 호출할 수 있습니다.

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## 비 즉시 실행 계산 및 TensorFlow

두 가지 주요 제한 사항을 알고 있어야 합니다. 첫째, Python 인터프리터가 `tff.federated_computation` 데코레이터를 만나면 함수가 한 번 추적되고 나중에 사용할 수 있도록 직렬화됩니다. 페더레이션 학습의 분산된 특성으로 인해 이러한 미래의 사용은 원격 실행 환경과 같은 다른 곳에서 발생할 수 있습니다. 따라서 TFF 계산은 기본적으로 *비 즉시 실행*입니다. 이 동작은 TensorFlow의 [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) 데코레이터와 다소 유사합니다.

둘째, 페더레이션 계산은 페더레이션 연산자(예: `tff.federated_mean`)로만 구성될 수 있으며 TensorFlow 작업을 포함할 수 없습니다. TensorFlow 코드는 `tff.tf_computation` 데코레이션된 블록으로 제한되어야 합니다. 숫자를 취하고 여기에 `0.5`를 더하는 다음 함수와 같이 대부분의 일반적인 TensorFlow 코드는 직접 데코레이션할 수 있습니다.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

또한 유형 서명이 있지만 *배치는 없습니다* . 예를 들어 다음을 호출할 수 있습니다.

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

여기에서 `tff.federated_computation`과 `tff.tf_computation`의 중요한 차이점을 볼 수 있습니다. 전자는 명시적 배치가 있는 반면 후자는 그렇지 않습니다.

게재 위치를 지정하여 페더레이션 계산에서 `tff.tf_computation` 블록을 사용할 수 있습니다. 절반을 추가하지만 클라이언트에서 페더레이션 부동 소수점에만 추가하는 함수를 만들어 보겠습니다. 게재 위치를 유지하면서 지정된 `tff.tf_computation`을 적용하는 `tff.federated_map`을 사용하여 이를 수행할 수 있습니다.

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

이 함수는 `tff.CLIENTS`에 배치된 값만 허용하고 동일한 배치의 값을 반환한다는 점을 제외하면 `add_half`와 거의 동일합니다. 유형 서명에서 이를 확인할 수 있습니다.

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

요약하자면:

- TFF는 페더레이션 값에서 작동합니다.
- 각 페더레이션 값에는 *유형*(예: `tf.float32`)과 *배치*(예: `tff.CLIENTS`)가 있는 *페더레이션 유형{/em}이 있습니다.*
- 페더레이션 값은 *페더레이션 계산*을 사용하여 변환될 수 있으며, `tff.federated_computation` 및 연합 유형 서명으로 데코레이션되어야 합니다.
- TensorFlow 코드는 `tff.tf_computation` 데코레이터가 있는 블록에 포함되어야 합니다.
- 그런 다음 이러한 블록을 페더레이션 계산에 통합할 수 있습니다.


# 고유한 페더레이션 학습 알고리즘 구축하기, 재검토

이제 페더레이션 코어를 살펴보았으므로 자체적인 페더레이션 학습 알고리즘을 구축할 수 있습니다. 위에서 알고리즘에 대해 `initialize_fn` 및 `next_fn`을 정의했음을 상기하세요. `next_fn`은 순수한 TensorFlow 코드를 사용하여 정의한 `client_update` 및 `server_update`를 사용합니다.

그러나 알고리즘을 페더레이션 계산으로 만들려면 `next_fn`과 `initialize_fn`이 각각 `tff.federated_computation`이 되어야 합니다.

## TensorFlow 페더레이션 블록 

### 초기화 계산 만들기

initialize 함수는 매우 간단합니다. `model_fn`을 사용하여 모델을 생성합니다. 그러나 `tff.tf_computation`을 사용하여 TensorFlow 코드를 분리해야 합니다.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

그런 다음 `tff.federated_value`를 사용하여 이를 페더레이션 계산에 직접 전달할 수 있습니다.

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### `next_fn` 만들기

이제 클라이언트 및 서버 업데이트 코드를 사용하여 실제 알고리즘을 작성합니다. 먼저 `client_update`를 클라이언트 데이터세트와 서버 가중치를 받아들이고 업데이트된 클라이언트 가중치 텐서를 출력하는 `tff.tf_computation`으로 변환합니다.

함수를 적절하게 데코레이션하려면 해당 유형이 필요합니다. 다행히도 서버 가중치 유형은 모델에서 직접 추출할 수 있습니다.

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

데이터세트 유형 서명을 살펴보겠습니다. 28 x 28 이미지(정수 레이블 포함)를 가져와 병합했다는 사실을 상기하세요.

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

위의 `server_init` 함수를 사용하여 모델 가중치 유형을 추출할 수도 있습니다.

In [ ]:
model_weights_type = server_init.type_signature.result

형식 서명을 살펴보면 모델의 아키텍처를 볼 수 있습니다!

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

이제 클라이언트 업데이트를 위한 `tff.tf_computation`을 생성할 수 있습니다.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

서버 업데이트의 `tff.tf_computation` 버전은 이미 추출한 유형을 사용하여 유사한 방식으로 정의할 수 있습니다.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

마지막으로 이 모든 것을 통합하는 `tff.federated_computation`을 만들어야 합니다. 이 함수는 두 개의 *페더레이션 값*을 받아 들입니다. 하나는 서버 가중치(게재 위치 `tff.SERVER`)에 해당하고 다른 하나는 클라이언트 데이터세트(게재 위치 `tff.CLIENTS`)에 해당합니다.

이 두 유형은 모두 위에서 정의되었습니다! `tff.FederatedType`을 사용하여 적절한 배치를 제공하기만 하면 됩니다.

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

FL 알고리즘의 4가지 요소를 기억하십니까?

1. 서버-클라이언트 브로드캐스트 단계
2. 로컬 클라이언트 업데이트 단계
3. 클라이언트-서버 업로드 단계
4. 서버 업데이트 단계

이제 위의 내용을 작성했으므로 각 부분을 한 줄의 TFF 코드로 간결하게 표현할 수 있습니다. 이 단순함 때문에 페더레이션 유형과 같은 것을 지정하기 위해 특별히 주의해야 했습니다!

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

이제 알고리즘 초기화와 알고리즘의 한 단계를 실행하기 위한 `tff.federated_computation`이 있습니다. 알고리즘을 완료하기 위해 이를 `tff.templates.IterativeProcess`에 전달합니다.

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

반복 프로세스의 <code>initialize</code> 및 `next` 함수의 <em>유형 서명</em>을 살펴보겠습니다.

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

이는 `federated_algorithm.initialize`가 단일 레이어 모델(784x10 가중치 행렬 및 10개의 바이어스 단위 포함)을 반환하는 인수가 없는 함수라는 사실을 반영합니다.

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

여기에서 `federated_algorithm.next`는 서버 모델과 클라이언트 데이터를 받아들이고 업데이트된 서버 모델을 반환합니다.

## 알고리즘 평가

몇 라운드를 실행하고 손실이 어떻게 변하는지 살펴보겠습니다. 먼저 두 번째 튜토리얼에서 논의한 *중앙 집중식* 접근 방식을 사용하여 평가 기능을 정의합니다.

먼저 중앙 집중식 평가 데이터세트를 만든 다음 훈련 데이터에 사용한 것과 동일한 전처리를 적용합니다.

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

다음으로 서버 상태를 받아들이고 Keras를 사용하여 테스트 데이터세트를 평가하는 함수를 작성합니다. `tf.Keras`에 익숙하다면 모두 낯설지 않겠지만 `set_weights`의 사용에 주목하세요!

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

이제 알고리즘을 초기화하고 테스트세트에서 평가해 보겠습니다.

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


몇 라운드 동안 훈련하고 변경 사항이 있는지 살펴보겠습니다.

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


손실 함수가 약간 감소하는 것을 볼 수 있습니다. 점프는 작지만 소수의 클라이언트에 대해 15번의 훈련 라운드만 수행했습니다. 더 나은 결과를 보려면 수천 번은 아니더라도 수백 번은 수행해야 할 수도 있습니다.

## 알고리즘 수정

이 시점에서 멈추고 우리가 성취한 것에 대해 생각해 봅시다. 순수 TensorFlow 코드(클라이언트 및 서버 업데이트용)를 TFF의 Federated Core 페더레이션 계산과 결합하여 Federated Averaging을 직접 구현했습니다.

보다 정교한 학습을 수행하려면 위에 있는 내용을 간단히 변경할 수 있습니다. 특히 위의 순수 TF 코드를 편집하여 클라이언트가 학습을 수행하는 방법 또는 서버가 모델을 업데이트하는 방법을 변경할 수 있습니다.

**도전 과제:** <code>client_update</code> 함수에 <a>그레디언트 클리핑</a>을 추가합니다.


더 큰 변경을 원하면 서버가 더 많은 데이터를 저장하고 브로드캐스트하도록 할 수도 있습니다. 예를 들어, 서버는 클라이언트 학습률을 저장하고 시간이 지남에 따라 감소하게 만들 수도 있습니다! 이를 위해 위의 `tff.tf_computation` 호출에 사용된 유형 서명을 변경해야 합니다.

**더 어려운 과제:** 클라이언트에서 학습률 감소를 사용하여 Federated Averaging을 구현합니다.

이 시점에서 이 프레임워크에서 구현할 수 있는 유연성이 얼마나 되는지 깨닫기 시작할 수 있습니다. 아이디어(위의 더 어려운 과제에 대한 답변 포함)가 필요하면 [`tff.learning.build_federated_averaging_process`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/build_federated_averaging_process)에 대한 소스 코드를 보거나 TFF를 사용하는 다양한 [연구 프로젝트](https://github.com/google-research/federated)를 확인할 수 있습니다.